In [29]:
import pymongo
import pandas as pd

In [30]:
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [39]:
def toGeoJSON(lista):
    return {
            "type": "Point",
            "coordinates":[lista[1],lista[0]]
        }

def getValuatio(funding_round):
    valor = 0
    for lista in funding_round:
        valor = lista['raised_amount']
    return valor

    


def convertRestaurant(company):
    lista_dict = []
    for index in range(len(company['offices'])):
        for key in company['offices']:
            if key['latitude'] != None and key['longitude'] != None:
                comp_off = {
                "name": company["name"],
                "category_code": company["category_code"],
                "number_of_employees": company["number_of_employees"],
                "founded_year": company["founded_year"],
                "valuation": getValuatio(company['funding_rounds']),
                "position": toGeoJSON([key['latitude'],key['longitude']])   
                }
                lista_dict.append(comp_off)
        return lista_dict

cursor = db.companies.find({"$and": [ { "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }, { "founded_year": {"$gt": 1995} }, {"deadpooled_year": None}  ] },{"_id":0})
ofiComp = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies = [element for lista in ofiComp for element in lista]


In [42]:
df = pd.DataFrame(geoCompanies)

df.head()
   


category_code           object
founded_year             int64
name                    object
number_of_employees    float64
position                object
valuation              float64
dtype: object

In [55]:
def getRatingCode(code):
    
    ratingCode = 0
    if code == ("web" or "software" or "hardware" or "design" or "consulting"):
        ratingCode += 5
    elif code == ("advertising" or "games_video" or  "ecommerce"):
        ratingCode += 4
    elif code == ("network_hosting" or "search" or "cleantech" or "analytics" or "messaging"):
        ratingCode += 3
    elif code == ("mobile" or "public_relations"  or "finance" or "fashion"):
        ratingCode += 2
    elif code == ("biotec" or "security" or "semiconductor" or "education" or "medical" or "health" or "manufacturing" or "nanotech" or "social" or "music" or "news" or "travel" or "photo_video"):    
        ratingCode += 1
    else:
        ratingCode += 0
    return ratingCode

ratingCode = df['category_code'].map(getRatingCode)
    


In [54]:
def getRatingFunds(funds):
    ratingFunds = 0
    if funds >= 100000000:
        ratingFunds += 5
    elif funds >= 8000000 and funds < 100000000:
        ratingFunds += 4
    elif funds >= 3000000 and funds < 8000000:
        ratingFunds += 3
    elif funds >= 1500000 and funds < 3000000:
        ratingFunds += 2
    elif funds >= 500000 and funds < 1500000:
        ratingFunds += 1
    else:
        ratingFunds += 0
    return ratingFunds


ratingFunds = df['valuation'].map(getRatingFunds)    


In [ ]:
def getRatingEmpl(employees):
    ratingEmpl = 0
    if employees >= 50000:
        ratingEmpl += 5
    elif employees >= 10000 and employees < 50000:
        ratingEmpl += 4
    elif employees >= 1000 and employees < 10000:    
        ratingEmpl += 3
    elif employees >= 200 and employees < 1000:    
        ratingEmpl += 2
    elif employees >= 50 and employees < 200:    
        ratingEmpl += 1
    else:    
        ratingEmpl += 0
    return ratingEmpl

ratingEmpl = df['number_of_employees'].map(getRatingEmpl)   

    
    


In [60]:
def getRatingYear(year):        
    ratingYear = 0
    if year > 2010:
        ratingYear += 5
    return ratingYear

ratingYear = df['founded_year'].map(getRatingYear)


In [62]:
def rating(df):
    return df['number_of_employees'].map(getRatingEmpl) + df['valuation'].map(getRatingFunds) + df['category_code'].map(getRatingCode) + df['founded_year'].map(getRatingYear)

rating = rating(df)

In [63]:
df['rating'] = rating

In [64]:
df.head()

,category_code,founded_year,name,number_of_employees,position,valuation,rating
0,news,2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5000000.0,4
1,web,1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",0.0,5
2,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",5000000.0,8
3,web,2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",5300000.0,8
4,web,2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",NaN,5


In [65]:
df.to_json("comp_with_coords.json", orient="records", lines=True)

In [5]:
cursor = db.location.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-122, 40.7046139]
      },
      "$minDistance": 0,
      "$maxDistance":1000000
    }
  }
})



In [6]:
df = pd.DataFrame(cursor)
df

,_id,category_code,founded_year,name,number_of_employees,position,valuation
0,5c69ad392918d50b1381f746,software,2001,Altivation Software,NaN,"{'type': 'Point', 'coordinates': [-121.815156,...",0.0
1,5c69ad392918d50b1381e5d3,web,2007,Design by Humans,15.0,"{'type': 'Point', 'coordinates': [-121.8211856...",0.0
2,5c69ad392918d50b1381e4e6,web,2003,Sellathon,NaN,"{'type': 'Point', 'coordinates': [-121.835193,...",0.0
3,5c69ad392918d50b1381e96b,ecommerce,1998,Auctiva,80.0,"{'type': 'Point', 'coordinates': [-121.835193,...",0.0
4,5c69ad392918d50b1381eb35,web,2008,NewDocs,3.0,"{'type': 'Point', 'coordinates': [-122.6883052...",0.0
5,5c69ad392918d50b1381edba,network_hosting,2000,StreamGuys,19.0,"{'type': 'Point', 'coordinates': [-124.0625123...",0.0
6,5c69ad392918d50b1381e1af,public_relations,2007,Global Business Development Services,6.0,"{'type': 'Point', 'coordinates': [-122.7032759...",0.0
7,5c69ad392918d50b1381e125,advertising,2007,New Leaders,6.0,"{'type': 'Point', 'coordinates': [-120.1994216...",0.0
8,5c69ad392918d50b1381dba7,web,2006,Diigo,NaN,"{'type': 'Point', 'coordinates': [-119.81439, ...",0.0
9,5c69ad392918d50b1381dd37,web,2007,LittleEngine,5.0,"{'type': 'Point', 'coordinates': [-121.234636,...",0.0


In [ ]:
cursor = db.clean.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-74.1080159, 40.7046139]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})

import pandas as pd
df = pd.DataFrame(list(cursor))
df.head()

In [ ]:



category_code
deadpooled_year
founded_year
funding_rounds     raised_amount
number_of_employees
offices      latitude
            longitude
    
    "offices": 
        deadpoolyear: None
            
            
def toGeoJSON(lista):
    for i in lista:
        return {
            "type":"Point",
            "coordinates":(i['longitude'],i['latitude'])
        }

def convertRestaurant(company):
    for diccio in company["offices"]:
            
        return {
        "name": company["name"],
        "homepage_url": company["homepage_url"],
        "twitter_username": company["twitter_username"],
        "category_code": company["category_code"],
        "number_of_employees": company["number_of_employees"],
        "founded_year": company["founded_year"],     
        "valuation": company["ipo"]['valuation_amount']     
        "position": toGeoJSON(company["offices"])
    }

cursor = db.companies.find({"name": "AddThis"}).limit(5)
geoCompanies = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies
{$and: [ { "offices.latitude": {"$exists": true} },{ "offices.longitude": {"$exists": true} } ] }
{$and: [ { "offices.latitude": {"$exists": true} },{ "offices.longitude": {"$exists": true} }, { founded_year: {"$gt": 1995} } ] }

5    web, software, hardware, design, consulting       
4    advertising, games_video, ecommerce,  
3    network_hosting, search, cleantech, analytics, messaging,
2    mobile, public_relations, finance, fashion
1    biotec, security, semiconductor, education, medical, health, manufacturing, nanotech, social, music, news, travel, photo_video
0    else

5                 más de 100 mill
4                  de 8 a 100 mill
3                 3 a 8 mill
2                 1,5 a 3 mill
1                 0,5  a 1,5
0                 else

5     más de 50000
4     10000 a 50000
3     1000 a 10000
2     200 a 1000
1     50 a 200
else  0    